In [1]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from matplotlib.colors import ListedColormap
from sklearn import datasets #Import scikit-learn dataset library
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import GridSearchCV # Import for hyperparameter tuning
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.metrics import confusion_matrix
from torchvision.transforms import v2
from torchvision import tv_tensors


In [ ]:
#List images and subfolders and label files

import os

images_path = '/content/drive/MyDrive/DSBA 6156/DSBA 6156 Final Project/Data/images'
labels_path = '/content/drive/MyDrive/DSBA 6156/DSBA 6156 Final Project/Data/labels'

all_image_entries = os.listdir(images_path)
image_subfolder_paths = [os.path.join(images_path, entry) for entry in all_image_entries if os.path.isdir(os.path.join(images_path, entry))]

all_label_entries = os.listdir(labels_path)
label_csv_paths = [os.path.join(labels_path, entry) for entry in all_label_entries if entry.endswith('.csv')]

print("Image Subfolder Paths:")
print(image_subfolder_paths[:5])
print("\nLabel CSV Paths:")
print(label_csv_paths[:5])

In [ ]:
# Extract identifiers and remove csv extension

image_identifiers = []
for path in image_subfolder_paths:
    identifier = os.path.basename(path)
    image_identifiers.append(identifier)

label_identifiers = []
for path in label_csv_paths:
    filename = os.path.basename(path)
    identifier = os.path.splitext(filename)[0]
    label_identifiers.append(identifier)

print("First 5 Image Identifiers:")
print(image_identifiers[:5])
print("\nFirst 5 Label Identifiers:")
print(label_identifiers[:5])

In [ ]:
# Create mapping

image_label_map = {}
matched_image_identifiers = set()

for label_path, label_identifier in zip(label_csv_paths, label_identifiers):
    # Find a matching image identifier
    matching_image_identifier = None
    # Iterate through image identifiers to find a match.
    for image_identifier in image_identifiers:
        if label_identifier in image_identifier:
            matching_image_identifier = image_identifier
            break

    if matching_image_identifier:
        image_label_map[matching_image_identifier] = label_path
        matched_image_identifiers.add(matching_image_identifier)
    else:
        print(f"No matching image subfolder found for label identifier: {label_identifier}")

# Identify unmatched image identifiers
all_image_identifiers_set = set(image_identifiers)
unmatched_image_identifiers = list(all_image_identifiers_set - matched_image_identifiers)

if unmatched_image_identifiers:
    print("\nUnmatched Image Identifiers (no corresponding label file):")
    for identifier in unmatched_image_identifiers:
        print(identifier)
else:
    print("\nAll image identifiers have a corresponding label file.")

# Convert to DataFrame
df_image_labels = pd.DataFrame.from_dict(image_label_map, orient='index').reset_index()
df_image_labels.columns = ['Image Identifier', 'Label Filename']

# Add 'Label Data' column with a placeholder, will be "None" for now
df_image_labels['Label Data'] = None

display(df_image_labels.head())

In [ ]:
#Iterate through images folers and label csv files

image_label_pairs = []
for index, row in df_image_labels.iterrows():
    pair = {
        'Image Identifier': row['Image Identifier'],
        'Label Filename': row['Label Filename']
    }
    image_label_pairs.append(pair)

print(f"Extracted {len(image_label_pairs)} image-label pairs.")
if image_label_pairs:
    print("First 5 extracted pairs:")
    for i, pair in enumerate(image_label_pairs[:5]):
        print(f"Pair {i+1}: {pair}")
else:
    print("No pairs were extracted.")

In [ ]:
#Read label csv for each mapped pair, put into dataframe

for pair in image_label_pairs:
    try:
        label_data_df = pd.read_csv(pair['Label Filename'])
        pair['Label Data'] = label_data_df
    except Exception as e:
        print(f"Error reading {pair['Label Filename']}: {e}")
        pair['Label Data'] = None # Assign None in case reading fails

if image_label_pairs:
    print("First 5 pairs with loaded label data:")
    for i, first_item in enumerate(image_label_pairs[:5]):
        print(f"Pair {i+1}:")
        print(f"  Image Identifier: {first_item['Image Identifier']}")
        print(f"  Label Filename: {first_item['Label Filename']}")
        print("  Label Data (first 5 rows):")
        if first_item['Label Data'] is not None:
            display(first_item['Label Data'].head())
        else:
            print("    Failed to load data.")
        print("-" * 20)

In [ ]:
#Extract image filenames and labels from label dataframe

for pair in image_label_pairs:
    if pair['Label Data'] is not None:
        extracted_labels = []
        for index, row in pair['Label Data'].iterrows():
            extracted_labels.append({
                'image_filename': row['id'],
                'label': row['label']
            })
        pair['Extracted Labels'] = extracted_labels
    else:
        pair['Extracted Labels'] = [] # Assign empty list in case data wasn't loaded

if image_label_pairs:
    print("First 5 pairs with extracted labels:")
    for i, pair in enumerate(image_label_pairs[:5]):
        print(f"Pair {i+1}:")
        print(f"  Image Identifier: {pair['Image Identifier']}")
        print(f"  Label Filename: {pair['Label Filename']}")
        print("  Extracted Labels (first 5 entries):")
        if pair['Extracted Labels']:
            for label_entry in pair['Extracted Labels'][:5]:
                print(f"    {label_entry}")
        else:
            print("    No labels extracted or data was not loaded.")
        print("-" * 20)

In [ ]:
#Create image file paths for each image filename extracts from csv files

for pair in image_label_pairs:
    if pair['Extracted Labels']:
        image_identifier = pair['Image Identifier']
        for label_entry in pair['Extracted Labels']:
            image_filename = label_entry['image_filename']
            full_image_path = os.path.join(images_path, image_identifier, image_filename)
            label_entry['image_path'] = full_image_path
    else:
        print(f"No extracted labels for {pair['Image Identifier']}")

if image_label_pairs:
    print("\nFirst 5 pairs with constructed image paths:")
    for i, pair in enumerate(image_label_pairs[:5]):
        print(f"Pair {i+1}:")
        print(f"  Image Identifier: {pair['Image Identifier']}")
        print("  Extracted Labels (first 5 entries with image_path):")
        if pair['Extracted Labels']:
            for label_entry in pair['Extracted Labels'][:5]:
                print(f"    {label_entry}")
        else:
            print("    No labels extracted or data was not loaded.")
        print("-" * 20)


In [ ]:
# Combine image path with labels extracted from csv files

combined_data_list = []

for pair in image_label_pairs:
    if pair.get('Extracted Labels'):
        for label_entry in pair['Extracted Labels']:
            combined_data_list.append({
                'image_path': label_entry.get('image_path'),
                'label': label_entry.get('label')
            })

df_combined_image_labels = pd.DataFrame(combined_data_list)

display(df_combined_image_labels.head())

In [ ]:
# Summarize Labels with Counts of each to visualize how many pictures we have of each position

label_counts = df_combined_image_labels['label'].value_counts().reset_index()
label_counts.columns = ['Label', 'Count']

print("Summary Table of Labels:")
display(label_counts)

In [ ]:
# More visualization

plt.figure(figsize=(10, 6))
sns.barplot(x='Label', y='Count', data=label_counts, palette='viridis')
plt.title('Distribution of Image Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Filter out undefined data

df_filtered_labels = df_combined_image_labels[df_combined_image_labels['label'] != 'undefined'].copy()

print("Filtered DataFrame (first 5 rows):")
display(df_filtered_labels.head())

print(f"\nNumber of rows in the filtered DataFrame: {len(df_filtered_labels)}")

In [ ]:
# Define your label as numbers
label_mapping = {'sitting': 0, 'standing': 1, 'lying': 2}

# 2. Create a custom Dataset
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # Open and convert to RGB
        img = Image.open(image_path).convert("RGB")

        # Apply transforms (resize, normalize, etc.)
        if self.transform:
            img = self.transform(img)

        # Convert label to tensor
        label_num = torch.tensor(label_mapping[label], dtype=torch.long)

        return img, label_num

# 3. Define transforms (resize → tensor → normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # resize
    transforms.ToTensor(),           # convert to tensor [0,1]
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

# Take a sample of the data
sample_size = 1000
df_sampled_labels = df_filtered_labels.sample(n=sample_size, random_state=42).reset_index(drop=True)


# 4. Create dataset using the sampled data
dataset = CustomImageDataset(
    image_paths=df_sampled_labels['image_path'].tolist(),
    labels=df_sampled_labels['label'].tolist(),
    transform=transform
)

# 5. Create DataLoader (handles batching + parallel loading)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

# 6. Iterate through batches (optional, just for checking)
for images, labels in dataloader:
    print("Batch of images:", images.shape)   # e.g., torch.Size([32, 3, 224, 224])
    print("Batch of labels:", labels.shape)   # e.g., torch.Size([32])
    break

In [ ]:
# Split the dataset into training and testing sets
train_dataset, test_dataset = train_test_split(dataset, test_size=0.3, random_state=42)

# Split the testing set into validation and test sets
val_dataset, test_dataset = train_test_split(test_dataset, test_size=0.5, random_state=42)

print(f"Number of samples in training set: {len(train_dataset)}")
print(f"Number of samples in validation set: {len(val_dataset)}")
print(f"Number of samples in test set: {len(test_dataset)}")

In [ ]:
# Create DataLoader for the test set
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

print(f"Test DataLoader created with {len(test_dataloader)} batches.")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.flatten = nn.Flatten()
        # Calculate the output size of the convolutional layers
        # Assuming input image size is 224x224 after preprocessing
        # After conv1 and pool1: (224 / 2) x (224 / 2) = 112 x 112, 16 channels
        # After conv2 and pool2: (112 / 2) x (112 / 2) = 56 x 56, 32 channels
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.relu3(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
num_classes = len(label_mapping)  # Get the number of unique labels
model = SimpleCNN(num_classes)

print("Simple CNN model defined:")
print(model)

In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adjust learning rate as needed

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create DataLoaders for training and validation sets
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)


# Training loop
num_epochs = 10  # Adjust the number of epochs as needed

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    for images, labels in train_dataloader:
        # Move data to GPU if available
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images) # Forward pass
        loss = criterion(outputs, labels) # Calculate the loss
        loss.backward() # Backpropagate the gradients
        optimizer.step() # Update model weights

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    # Validation (optional, but good practice)
    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad(): # Disable gradient calculation for validation
        for images, labels in val_dataloader:
            # Move data to GPU if available
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")

print("Training finished.")

In [ ]:
# Evaluate the model on the test set
model.eval()  # Set model to evaluation mode
correct = 0
total = 0
all_predicted = []
all_labels = []

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = 100 * correct / total
print(f"Accuracy on the test set: {accuracy:.2f}%")

# Generate confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predicted)

print("\nConfusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_mapping.keys(), yticklabels=label_mapping.keys())
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()